In [1]:
from multiprocessing import Pool
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import os
from time import time
import time
import pickle
import csv

np.set_printoptions(precision=2)

import sys
sys.path.append("../modules/")
import utils
import unbiased_estimation

import imp
imp.reload(unbiased_estimation)
imp.reload(utils)

<module 'utils' from '../modules/utils.py'>

# Utilities for sampling

In [ ]:
def plot_clusts(data, clusts, ax=None):
    if ax == None:
        ax = plt.subplot(111)
        show=True
    else:
        show = False
        
    for idcs in clusts:
        idcs = list(idcs)
        x_clust = data[idcs]
        ax.scatter(x_clust[:, 0], x_clust[:, 1])
    lim = np.max(np.abs(data))
    ax.set_xlim([-lim,lim])
    ax.set_ylim([-lim,lim])
    
    if show: plt.show()

In [ ]:
# data structure for state
# z : N-vector of assignments
# clusts : list of lists of cluster assignments
# for each cluster c, z[clusts[c]] is a vector of c's

def pop_idx(z, clusts, n):
    """pop_idx removes the datapoint n from the current cluster assignments
    
    Args:
        z: list of cluster assignments
        clusts: list clusters (each cluster is a set)
        n: index of datapoint to remove
    
    Returns:
        updated labels, clusters, and index of removed cluster (or None if no cluster was removed)
    
    
    """
    c = z[n]
    z[n] = -1
    clusts[c].remove(n)
    
    # if cluster is empty, remove it and reassign others
    if len(clusts[c]) == 0:
        removed_cluster = c
        for i in range(c, len(clusts)-1):
            clusts[i] = clusts[i+1]
            z[list(clusts[i])] = i
        clusts.pop() # drop last cluster
    else:
        removed_cluster = None
    return z, clusts, removed_cluster

In [ ]:
def gibbs_sweep_single(data, z, sd, sd0, alpha=0.01):
    clusts = utils.z_to_clusts(z)  # initial data counts at each cluster
    # take a Gibbs step at each data point
    for n in range(len(data)):
        # get rid of the nth data point and relabel clusters if needed
        z, clusts, _ = pop_idx(z, clusts, n)

        # sample which cluster this point should belong to
        loc_probs = utils.weights_n(data, clusts, sd, sd0, alpha, n)
        newz = np.random.choice(len(loc_probs), p=loc_probs)

        # if necessary, instantiate a new cluster
        if newz == len(clusts): clusts.append(set())

        # update cluster assigments
        clusts[newz].add(n)
        z[n] = newz

    return z

def crp_gibbs(data, sd, sd0, initz, alpha=0.01, plot=True, log_freq=None, maxIters=100):
    """crp_gibbs runs a gibbs sampler 
    
    The state of the chain includes z and clusts, which are redundant 
    but make for easy / fast indexing.
    
    just setting alpha for inference.
    a small alpha encourages a small number of clusters.
    
    Returns:
        z  (array of cluster assignments) and clusts (list of clusters)
    
    """

    # initialize the sampler
    z = np.array(initz) # don't overwrite
    clusts = utils.z_to_clusts(z)  # initial data counts at each cluster
    
    # set frequency at which to log state of the chain
    if log_freq is None: log_freq = int(maxIters/10)
    
    # run the Gibbs sampler
    for I in range(maxIters):
        # take a Gibbs step at each data point
        z = gibbs_sweep_single(data, z.copy(), sd, sd0, alpha=alpha)
        
        if (I%log_freq==0 or I==maxIters-1) and plot:
            clusts = utils.z_to_clusts(z)  # initial data counts at each cluster
            print("Iteration %04d/%04d"%(I, maxIters))
            plot_clusts(data, clusts, ax=None)
    return z, utils.z_to_clusts(z)

In [ ]:
def gibbs_sweep_couple(data, z1, z2, sd, sd0, alpha=0.01, coupling="Maximal"):
    """gibbs_sweep_couple performs Gibbs updates for every label in sequence, 
    coupling each update across the two chains.
    
    We compute intersection sizes once at the start and then update it for better time complexity.
    """
    # Compute clusters and intersection sizes from scratch once
    clusts1, clusts2 = utils.z_to_clusts(z1), utils.z_to_clusts(z2)  # initial data counts at each cluster
    intersection_sizes = np.array([[len(c1.intersection(c2)) for c2 in clusts2] for c1 in clusts1])
    
    # Take a Gibbs step at each data point
    for n in range(len(data)):
        # Get rid of the nth data point and relabel clusters if needed
        z1, clusts1, removed_clust1 = pop_idx(z1, clusts1, n)
        z2, clusts2, removed_clust2 = pop_idx(z2, clusts2, n)
        
        # Update intersection sizes with datapoint n removed
        if (removed_clust1 is None) and (removed_clust2 is None):
            # If neither cluster was removed, decrement size of intersection
            intersection_sizes[z1[n],z2[n]] -= 1
        else:
            # Otherwise remove corresponding row and or column
            if removed_clust1 is not None:
                intersection_sizes = np.delete(intersection_sizes, removed_clust1, axis=0)
            if removed_clust2 is not None:
                intersection_sizes = np.delete(intersection_sizes, removed_clust2, axis=1)
            
        # Compute marginal probabilities
        loc_probs1 = utils.weights_n(data, clusts1, sd, sd0, alpha, n)
        loc_probs2 = utils.weights_n(data, clusts2, sd, sd0, alpha, n)

        # Sample new clusters assignments from coupling
        if coupling=="Common_RNG":
            newz1, newz2 = utils.naive_coupling(loc_probs1, loc_probs2)
        elif coupling=="Maximal":
            newz1, newz2 = utils.max_coupling(loc_probs1, loc_probs2)
        else:
            assert coupling=="Optimal"
            pairwise_dists = utils.pairwise_dists(clusts1, clusts2, intersection_sizes)
            _, (newz1, newz2), _ = utils.optimal_coupling(
                loc_probs1, loc_probs2, pairwise_dists, normalize=True,
                change_size=100)

        # if necessary, instantiate a new cluster and pad intersection_sizes appropriately
        if newz1 == len(clusts1):
            clusts1.append(set())
            intersection_sizes = utils.pad_with_zeros(intersection_sizes, axis=0)
        if newz2 == len(clusts2):
            clusts2.append(set())
            intersection_sizes = utils.pad_with_zeros(intersection_sizes, axis=1)

        # update cluster assigments and intersection sizes
        clusts1[newz1].add(n); clusts2[newz2].add(n)
        z1[n], z2[n] = newz1, newz2
        intersection_sizes[z1[n],z2[n]] += 1
        
    return z1, z2

def crp_gibbs_couple(
    data, sd, sd0, initz1, initz2,alpha=0.01, plot=True,
    log_freq=None, maxIters=100, coupling="Maximal", save_base=None):
    """
    
    Args:
        coupling: method of coupling must be "Common_RNG", "Maximal" or "Optimal" ("Common_RNG" used to be "Naive")
    
    """
    
    # initialize the sampler
    z1, z2 = initz1, initz2
    z1s, z2s = [z1.copy()], [z2.copy()]
    
    dists_by_iter = []
    
    # set frequency at which to log state of the chain
    if log_freq is None: log_freq = int(maxIters/10)
    
    # run the Gibbs sampler
    for I in range(maxIters):
            
        z1, z2 = gibbs_sweep_couple(
            data, z1.copy(), z2.copy(), sd, sd0,
            alpha=alpha)
            
        # data counts at each cluster
        clusts1, clusts2 = utils.z_to_clusts(z1), utils.z_to_clusts(z2)  
        z1s.append(z1); z2s.append(z2)
        
        
        dist_between_partitions = utils.adj_dists_fast(clusts1, clusts2)
        dists_by_iter.append(dist_between_partitions)
        
        if (I%log_freq==0 or dist_between_partitions==0) and plot:
            print("Iteration %04d/%04d"%(I, maxIters))
            print("n_clusts: ", len(clusts1), len(clusts2))
            save_name = save_base + "_%04d.png"%I if save_base is not None else None
            plot_chains(data, clusts1, clusts2, save_name=save_name)
            
        if dist_between_partitions == 0:
            print("Chains coupled after %d iterations!"%I)
            break
        
    return z1, dists_by_iter

# Load data and set hyper-params

## Synthetic data

In [ ]:
def ex6_gen_data(Ndata, sd, sd0=1, K=2):
    # TRANSLATION OF TAMARA's CODE INTO PYTHON
    #
    # generate Gaussian mixture model data for inference later
    #
    # Args:
    #  Ndata: number of data points to generate
    #  sd: covariance matrix of data points around the
    #      cluster-specific mean is [sd^2, 0; 0, sd^2];
    #      i.e. this is the standard deviation in either direction
    #  sd0: std for prior mean
    #
    # Returns:
    #  x: an Ndata x 2 matrix of data points
    #  z: an Ndata-long vector of cluster assignments
    #  mu: a K x 2 matrix of cluster means,
    #      where K is the number of clusters

    # matrix of cluster centers: one in each quadrant
    mu = np.random.normal(scale=sd0, size=[K, 2])
    # vector of component frequencies
    #rho = np.array([0.4,0.3,0.2,0.1])
    rho = stats.dirichlet.rvs(alpha=2*np.ones(K))[0]

    # assign each data point to a component
    z = np.random.choice(range(K), p=rho, replace=True, size=Ndata)
    # draw each data point according to the cluster-specific
    # likelihood of its component
    x = mu[z] + np.random.normal(scale=sd, size=[Ndata,2])  
    
    return x

In [ ]:
"""
np.random.seed(44)
K = 3
Ndata = 50
sd, sd0, alpha = 2., 2., 0.5
data = ex6_gen_data(Ndata, sd, sd0, K=K)
"""

## Gene-expression data

In [ ]:
Ndata = 200
D = 50

sd, sd0, alpha = 2., 2., 0.5 # totally arbitrary
data = np.loadtxt('data/gene_data_Ndata=%d_D=%d.csv' %(Ndata,D),delimiter=',',skiprows=1,usecols=range(1,D+1))
print(data.shape)

# Define functions acting on partition

## Predictive density 

In [ ]:
# Unbiased estimation
def posterior_predictive_density(data, z, data_new, sd, sd0, alpha):
    """posterior_predictive_density computes the posterior predictive density, conditioned 
    on z and data at data_new (for the DP-mixture model).
    
    Args:
        data: observations
        z: assignments
        data_new: location of single observation at which to compute posterior predictive
        sd, sd0: standard deviation of noise and prior on means
        alpha: DP paramter
        
    Returns:
        posterior predictive density at data_new
    """
    Ndata = len(z)
    Ndata_new, D = data_new.shape
    
    # Initialize Posterior Predictive Density (ppd) as zeros 
    ppd = np.zeros(Ndata_new)
    
    # Add density corresponding to new cluster
    prob_new_cluster = alpha / (Ndata+alpha)
    ppd += prob_new_cluster * stats.multivariate_normal(
        mean=np.zeros(D), cov=(sd**2 + sd0**2)*np.eye(D)).pdf(data_new)
    
    for clust in set(np.unique(z)):
        # pull out data in cluster 'clust'
        data_c = data[np.where(z==clust)[0]]
        
        # probability that new datapoint is in clust
        prob_clust = len(data_c)/(Ndata + alpha)
        
        # posterior over mean of clust
        post_prec = (1./(sd0**2)) + len(data_c)*(1./(sd**2))
        post_var = 1./post_prec
        post_mean = post_var*(len(data_c)*(1./(sd**2))*np.mean(data_c,axis=0))
        
        ppd += prob_clust * stats.multivariate_normal(
            mean=post_mean, cov=(post_var + sd**2)*np.eye(D)).pdf(data_new)
    return ppd

def posterior_predictive_density_grid(data, z, sd, sd0, alpha, n_grid_spaces=20):
    """posterior_predictive_density_grid evaluate the posterior predictive density at a grid of points.
    
    Args:
        (Same as posterior_predictive_density)
        n_grid_spaces: granularity of locations at which to compute density
    """
    scale = 2*sd**2 + 2*sd0**2
    delta = scale/10
    
    # Create vector of locations at which to query predictive distribution
    x, y = np.arange(-scale, scale, delta), np.arange(-scale, scale, delta)
    X, Y = np.meshgrid(x, y)
    X_long, Y_long = X.reshape([-1]), Y.reshape([-1])
    data_new = np.array(list(zip(X_long, Y_long)))
    
    # Compute predictive density and reshape into a grid for easy visualization
    ppd = posterior_predictive_density(data, z, data_new, sd, sd0, alpha)
    ppd_grid = ppd.reshape(X.shape)
    X_Y_ppd = np.array([X, Y, ppd_grid])
    return X_Y_ppd

def plot_density(X, Y, Z, title=None):
    """plot_density shows a predictive density with a contour plot.
    
    Args:
        X, Y: x and y positions of density values, both np.array of shape [n_grid_spaces, n_grid_spaces]
        Z: density values
    
    """
    fig, ax = plt.subplots()
    CS = ax.contour(X, Y, Z)
    ax.clabel(CS, inline=1, fontsize=10)
    ax.set_title(title)
    plt.show()

## Top K proportions

In [ ]:
def prop_in_k_clusters(z, k=10):
    # compute cluster sizes in decreasing order.
    clusts = utils.z_to_clusts(z)
    clust_sizes = list(sorted([len(clust) for clust in clusts], reverse=True))
    
    # compute proportion of datapoints assigned to up to top k clusters.
    props = np.ones(shape=[k])
    props[:len(clust_sizes)] = np.cumsum(clust_sizes)[:k]/len(z)
    return props

In [ ]:
z = np.array([0,0,0,0,1,1,1,2,2])
clusts = utils.z_to_clusts(z)
clusts_as_lists = [list(clust) for clust in clusts]
clusts_sizes_with_labels = [(len(clust),z[clust[0]]) for clust in clusts_as_lists]

print(clusts_as_lists)
print(len(clusts_as_lists[0]))
print(clusts_sizes_with_labels)

## Co-clustering matrix
- a thin wrapper around utils.adj_matrix since adjacency matrix AKA co-clustering matrix

In [ ]:
def co_cluster(z):
    """
    Input:
        z: (N,) vector of labels
    Output:
        mat: (N,N) matrix of whether two observations are in the same cluster
    """
    clusts = utils.z_to_clusts(z)
    mat = utils.adj_matrix(z, clusts)
    return mat

# Define utilities for performing unbiased estimation

In [ ]:
def crp_prior(N,alpha):
    """
    There's got to be a simpler way to do this.
    
    Input:
        N: number of observations
        alpha: scalar, 
    
    Output:
        a sample from CRP prior
    """
    z0 = np.array([0])
    for n in range(1,N):
        # print(n)
        clusts = utils.z_to_clusts(z0)
        # print(clusts)
        clusts_as_lists = [list(clust) for clust in clusts]
        # print(clusts_as_lists)
        clusts_sizes_with_labels = [(len(clust),z0[clust[0]]) for clust in clusts_as_lists]
        probs_with_label = [(clust[0]/(n+alpha), clust[1]) for clust in clusts_sizes_with_labels] # adding to old clusters
        probs_with_label.append((alpha/(n+alpha),len(clusts))) # new cluster 
        probs = [x[0] for x in probs_with_label]
        # print(probs)
        new_point = probs_with_label[np.random.choice(len(probs), p=probs)][1]
        z0 = np.append(z0,new_point)
        # print(z0)
    return z0

def pi0(data, sd, sd0, alpha, pi0_its=0):
    """pi0 is the initial distribution for the Markov chain.
    """
    Ndata = data.shape[0]
    z0 = np.zeros(Ndata, dtype=np.int)
    for i in range(pi0_its): z0 = gibbs_sweep_single(data, z0.copy(), sd, sd0, alpha)
    return z0

def unbiased_est_crp(k, h, m, data, sd, sd0, alpha, pi0_its=0, init_type="crp_prior", coupling="Optimal"):#
    """unbiased_est produces an unbiased estimate of a functional using the approach of Jacob 2020
    
    initializes from r_init
    
    Args:
        k: # burn-in iterations
        h: lambda function of interest (of labelings z)
        m: minimum iterations
        data: observations
        pi0_its: number of iterations to run before coupling
        coupling: either maximal or optimal
    
    Returns: 
        unbiased estimate, number of steps before meeting, states of either chain until meeting
    """
    # Define initial distribution
    if init_type == "pi0":
        pi0_dp = lambda : pi0(data, sd, sd0, alpha, pi0_its)
    elif init_type == "crp_prior":
        pi0_dp = lambda : crp_prior(data.shape[0], alpha)
    
    # Define marginal and coupled transitions
    gibbs_sweep = lambda z: gibbs_sweep_single(data, z.copy(), sd, sd0, alpha)
    coupled_gibbs_sweep = lambda z1, z2: gibbs_sweep_couple(
        data, z1.copy(), z2.copy(), sd, sd0, alpha, coupling=coupling)
    
    # Run coupled chains
    X, Y, tau = unbiased_estimation.run_two_chains(m, pi0_dp, gibbs_sweep, coupled_gibbs_sweep)
    
    # Compute unbiased estimate
    H_km = unbiased_estimation.unbiased_est(k, h, m, X, Y, tau)

    return H_km, tau, X, Y

# Usual MCMC estimate for comparison. Should we do thinning of the single ergodic average?
def usual_MCMC_est_crp(k, h, m, data, sd, sd0, alpha, init_type="crp_prior"):
    """
    Inputs:
        k: # burn-in iterations
        h: lambda function of interest (of labelings z)
        m: scalar, number of sweeps
        data: observations
        sd, sd0, alpha: hyper-params
    """
    if (init_type == "all_same"):
        X = [np.zeros(data.shape[0], dtype=int)]
    elif (init_type == "crp_prior"):
        X = [crp_prior(data.shape[0],alpha)]
    for _ in range(m): X.append(gibbs_sweep_single(data, X[-1].copy(), sd, sd0, alpha))
    ests = [h(x) for x in X[k:]]
    mean = np.mean(ests, axis=0)
    return mean

In [ ]:
np.random.seed(4)

N = 200
alpha = 0.5

z = crp_prior(N,alpha)
print(utils.z_to_clusts(z))

# Estimation results

In [ ]:
def run_rep(est_type, h, maxIter, time_budget):
    """
    Only keep track of time to traverse the posterior, not time
    to actually evaluate the function.
    
    Input:
        est_type: "truth", "single", "coupled"
        h: lambda function
        maxIter: scalar 
        time_budget: scalar, 
        
    Return 
        states: list, only meaning full for the single or coupled estimators 
            (since we don't run for too long, not too memory intensive to save the Markov chains' 
            states)
        result: scalar, estimate of the integral of interest
        num_iter: number of estimators generated in the time_budget 
            (only interesting for est_type = "coupled")
    """
    np.random.seed() # seed = None actually means we use the most randomness across processors
    
    # don't care about time_budget, just sample for long to get estimate
    # of truth
    if (est_type == "truth"):
        result = usual_MCMC_est_crp(k, h, maxIter, data, sd, sd0, alpha,init_type)
        states = None
        num_sweeps = maxIter
        num_est = 1
        
    elif (est_type == "single"):
        if (init_type == "all_same"):
            X = [np.zeros(data.shape[0], dtype=int)]
        elif (init_type == "crp_prior"):
            X = [crp_prior(data.shape[0],alpha)]
        st = time.clock() # hopefully this is process-specific time
        num_sweeps = 0
        while (True):
            X.append(gibbs_sweep_single(data, X[-1].copy(), sd, sd0, alpha))
            num_sweeps += 1
            time_elasped = time.clock() - st 
            if (time_elasped >= time_budget):
                break
        # with reasonable time_budget, this shouldn't happen, but just in case
        if (len(X) <= k):
            states = None
            result = None
            num_est = 0
        else:
            ests = [h(x) for x in X[k:]]
            states = X 
            result = np.mean(ests, axis=0)
            num_est = 1
            
    # as long as time budget hasn't ended, keep generating unbiased estimators
    elif (est_type == "coupled"):
        states = [] # each entry will be a coupling attempt, with the states of either chain and the meeting time
        ests_ub = []
        num_sweeps = None
        num_est = 0
        st = time.clock() # hopefully this is process-specific time
        while (True):
            est, tau, X, Y = unbiased_est_crp(k, h, m, data, sd, sd0, alpha, pi0_its, init_type, coupling)
            if (time.clock() - st >= time_budget):
                break
            ests_ub += [est]
            states.append((X,Y,tau))
            num_est += 1
        # this is unlikely to happen if we set time_budget to be reasonable, but
        # just in case
        if (num_est == 0):
            result = None
        else:
            result = np.mean(ests_ub, axis=0)
    
    return states, result, num_sweeps, num_est

In [ ]:
h_type = "topK"
M = 10 # number of replicates 
time_budget = 500 # maximum time for each replicate
maxIter = 100
k = 10 # burn-in
m = 20 # minimum iterations
pi0_its = 5
pool_size = 4
init_type = "crp_prior"
coupling='Optimal'

## Top k proportions

In [ ]:
savedir = "estimation_results/gene_Ndata=%d_D=%d/" %(Ndata,D)
if not os.path.exists(savedir):
    print("Will make directory %s" %savedir)
    os.makedirs(savedir)
savedir = savedir + "topK_"
topK = 10
h = lambda z: prop_in_k_clusters(z, topK)

### Ground truth
- variation across M replicates is low

In [ ]:
est_type = "truth"
def simulate(_):
    result = run_rep(est_type, h, maxIter, time_budget)
    print("complete")
    return result
with Pool(pool_size) as p:
    results = p.map(simulate, range(M))
    
for res in results:
    print(res)
    
savepath = savedir + "truth_maxIter=%d.pkl" %(maxIter)
with open(savepath, 'wb') as f:
    pickle.dump(results, f)

### replicates of one chain time-budgetted run
- Ndata=500, D=150 each sweep takes ~ 500s processor time (after potential multi-threading). In wall-clock time it felt like only ~2 minutes.
- Ndata=200, D=50, in 300s processor time we can do ~ 20 sweeps

In [ ]:
est_type = "single"
def simulate(_):
    result = run_rep(est_type, h, maxIter, time_budget)
    print("complete")
    return result
with Pool(pool_size) as p:
    results = p.map(simulate, range(M))
    
for res in results:
    print(res)
    
savepath = savedir + "single_initType=%s_maxTime=%d_burnin=%d.pkl" %(init_type, time_budget,k)
with open(savepath, 'wb') as f:
    pickle.dump(results, f)

### replicates of coupled chains

In [ ]:
est_type = "coupled"
def simulate(_):
    result = run_rep(est_type, h, maxIter, time_budget)
    print("complete")
    return result
with Pool(pool_size) as p:
    coupled_results = p.map(simulate, range(M))
    
for res in coupled_results:
    print(res)
    
savepath = savedir + "%s_coupled_initType=%s_maxTime=%d_burnin=%d_minIter=%d.pkl" %(h_type, init_type, time_budget,k,m)
with open(savepath, 'wb') as f:
    pickle.dump(results, f)

## Co-clustering

In [ ]:
h = lambda z: co_cluster(z)

### Ground truth

In [ ]:
"""
pool_size = 3
def simulate(_):
    result = run_rep(est_type="truth", h, maxIter, time_budget)
    print("complete")
    return result
with Pool(pool_size) as p:
    results = p.map(simulate, range(M))
"""

### Replicates of one chain time-budgetted run

### Replicates of coupled chains